In [40]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
import copy
import json
%matplotlib inline

In [41]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = 'beep.wav'
# pd.set_option('display.max_colwidth', -1)

## Importing data

Importing a sample of the dataset:

In [42]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

Importing the whole dataset

In [43]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

filename = os.path.join('data','twex.tsv') # 'sample.tsv')
pr('Starting to read file... (3 min)')
# tw = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
#                       quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
# Audio(url=sound_file, autoplay=True)

19:03:17 Starting to read file... (3 min)
19:03:17 File is loaded!


In [44]:
tw.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,17341045.0,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586.0,508.0,9016.0,"Bern, Switzerland",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extrating hashtags

In [45]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    lowerCharList = [ht.lower() for ht in non_empty_hts]
    return lowerCharList

In [46]:
pr('Making hashtags... (2 min)')
tw['hashtag'] = np.nan
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)]
pr('Done.')

19:03:18 Making hashtags... (2 min)
19:03:39 Done.


In [47]:
twh.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,...,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
16,9519737890,14657884.0,2010-02-23 09:59:41,"Magic spells run off after midnight, I guess s...",6.1387,46.175,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[fb]
20,9521789689,9962022.0,2010-02-23 11:28:27,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[yam]


## Cleaning data and making date index

In [48]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 79200 tweets to 79200 tweets.


In [49]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

19:03:40 Removing bad dates...
19:03:40 Finished.


In [50]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

19:03:40 Starting to examine dates...
19:03:41 There are 0 dates that cannot be transformed.


In [51]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

19:03:41 Starting copy...
19:03:41 Converting to datetime...
19:03:41 Setting up new indices...
19:03:41 Deleting old "createdAt" column...
19:03:41 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,14657884.0,"Magic spells run off after midnight, I guess s...",6.1387,46.175,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[fb]
2010-02-23 11:28:27,9521789689,9962022.0,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[yam]


In [52]:
tw5['hashtag'][:6]

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11          [24, vfb]
2010-02-23 18:19:03    [iphoneography]
2010-02-23 18:31:46     [partnermonth]
2010-02-24 06:09:23      [insider, fb]
Name: hashtag, dtype: object

## Let's put one hashtag per row

We will make a dataframe with one row = one hashtag. This will be done by going through the dataframe, and making in parallel a list of rows (with 1 hashtag per row) that needs to be added to the old dataframe.

In [53]:
addedHashtagsRowsList = []
def multiplyHashtagRows(row, columns):
    '''
    Examine each row. If there are multiple hashtags, it will return the first one.
    (so the first one will replace the list of hashtags in the df). Then for all the next ones,
    it will make a copy of the row in the addedHashtagsRowsList, (in a dictionary format).
    So this dictionary can in the end be transformed in a DF and added to the original DF.
    (The speed is increased a lot by doing it this way!)
    '''
    htList = row.hashtag
    if len(htList) > 1:
        ## Making the dictionary
        addedHashtag = {}
        addedHashtag['createdAt'] = row.name #the df index
        for col in columns:
            addedHashtag[col] = row[col]
        ## Copying the dict for each hashtag
        i = 1
        while i < len(htList) :
            deepCopy = copy.deepcopy(addedHashtag)
            deepCopy['hashtag'] = htList[i]
            addedHashtagsRowsList.append(deepCopy)
            i+=1
    return htList[0] # return the first hashtag

In [54]:
addedHashtagsRowsList = []
tw5_1 = tw5.copy()
pr('Multiplying the hashtag rows... (around 10 min)')
tw5_1['hashtag'] = tw5.apply(multiplyHashtagRows, args=[tw5.columns,], axis=1)
pr('Finished! {} rows will be added to the dataframe!'.format(len(addedHashtagsRowsList)))

19:03:42 Multiplying the hashtag rows... (around 10 min)
19:04:15 Finished! 40780 rows will be added to the dataframe!


In [55]:
pr('Starting to make the new dataframe with additionnal rows..')
addedHashtagsDf = pd.DataFrame(addedHashtagsRowsList)
addedHashtagsDf.set_index(['createdAt'], inplace=True)
pr('Starting to append the two df... Old df size = {}'.format(len(tw5_1)))
tw6 = tw5_1.append(addedHashtagsDf)
pr('Done! New df size = {}'.format(len(tw6)))

19:04:15 Starting to make the new dataframe with additionnal rows..
19:04:16 Starting to append the two df... Old df size = 79162
19:04:16 Done! New df size = 119942


In [56]:
print('Example hahshtag:')
tw6[tw6['hashtag'] == addedHashtagsRowsList[0]['hashtag']].head(3)

Example hahshtag:


,followersCount,friendsCount,hashtag,id,inReplyTo,latitude,longitude,placeId,placeLatitude,placeLongitude,screenName,source,sourceName,sourceUrl,statusesCount,text,truncated,userId,userLocation,userName
createdAt,,,,,,,,,,,,,,,,,,,,
2011-07-24 16:03:13,NaN,NaN,vfb,95162125584039936,9.501397e+16,NaN,NaN,e401fb8eb4e7595a,NaN,NaN,NaN,14.0,NaN,NaN,NaN,"@mikstweed Wie lange ist es her, dass der #vfb...",NaN,921241.0,NaN,NaN
2010-02-23 17:47:11,NaN,NaN,vfb,9535390586,NaN,47.6463,9.1657,NaN,NaN,NaN,NaN,550.0,NaN,NaN,NaN,"So, Feierabend. Jetzt #24 und später #VfB. — a...",NaN,921241.0,NaN,NaN
2010-12-01 17:25:56,NaN,NaN,vfb,10021795398684672,NaN,46.9499,7.47071,e38a1a641d02f8db,NaN,NaN,NaN,1.0,NaN,NaN,NaN,"So, und jetzt Flachmann suchen und dann ab ins...",NaN,120433.0,NaN,NaN


In [57]:
tw6.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11               24
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

## Aggregation of rows by date

In [58]:
tw6.dropna(subset=['longitude'], inplace=True)
tw6.dropna(subset=['latitude'], inplace=True)

In [59]:
tw6.latitude = tw6.latitude.apply(float)
tw6.longitude = tw6.longitude.apply(float)

In [60]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

Function that applies to a dataframe will group each row by day and aggregate all its content:

In [61]:
def aggDate(df):
    groupedDf = df.groupby(df.index.map(lambda x: x.date)).agg({'text' : str_join,
                                                          'longitude' : np.median,
                                                          'latitude' : np.median,
                                                          'hashtag' : lambda x: x.iloc[0], ## the first occurence
                                                          'numberOfTweets' : 'count'})
    return groupedDf

In [62]:
pr('Making column number of tweets')
tw6['numberOfTweets'] = 1
pr('Starting group by hastag...')
gp = tw6.groupby('hashtag')
pr('Starting to put hashtag in dictionary... (around 50 min)')

count = 0
lengp = len(gp)
printingValue = int(lengp / 10)
dictionary = {}
for hashtag, df in gp:
    dictionary[hashtag] = aggDate(df)
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/lengp*100))
pr('Finished operations! Dictionary with {} different hashtags.'.format(len(dictionary)))

19:04:17 Making column number of tweets
19:04:17 Starting group by hastag...
19:04:17 Starting to put hashtag in dictionary... (around 50 min)
19:04:51 10%
19:05:23 20%
19:05:56 30%
19:06:28 40%
19:07:01 50%
19:07:31 60%
19:08:01 70%
19:08:35 80%
19:09:10 90%
19:09:42 100%
19:09:42 Finished operations! Dictionary with 27040 different hashtags.


In [63]:
# aggDate(gp.get_group('twibisg')).head(3)
# dictionary['twibisg'].head(3)
print('Dictionary with hashtags dataframes:')
dictionary[list(dictionary.keys())[5]].head(4)

Dictionary with hashtags dataframes:


,latitude,text,longitude,numberOfTweets,hashtag
2010-03-09,46.8819,Mmmh im #Tellbus mit #Maisbrötli — at Bus Halt...,8.64366,1,maisbrötli


## Starting event detection

In [64]:
## Parameters of an event:
MIN_TOT_NB_TWEETS = 20 ## The hashtag must have happened at least this number of times to be considered.
MIN_NB_DAYS_WITH_HASHTAGS = 3 ## The hashtags must appear at least this number of different days to be considered.
MIN_NB_TWEETS_DURING_EVENT = 5 ## To be considered an event, the hashtag must happen at least this nb of times during the day.
THRESHOLD_ANOMALY_FACTOR = 2 ## The occurence of a hashtag during a single day must be above the mean by this FACTOR
                             ## multiplied by the std to be considered as an anomaly.

In [65]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
            df['event'] = df.numberOfTweets.apply(lambda x: x > threshold and x > MIN_NB_TWEETS_DURING_EVENT)
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

19:09:42 Starting to compute 27040 dict items to detect event. (4 min)
19:09:45 10%
19:09:47 20%
19:09:50 30%
19:09:52 40%
19:09:55 50%
19:09:57 60%
19:10:00 70%
19:10:02 80%
19:10:04 90%
19:10:07 100%
19:10:07 Finished! Number of events detected = 260


## Making single event dataframe

In [66]:
eventRowsList = []
def applyToMakeEventDf(row):
    if row.event:
        rowToAdd = {'date': row.name, 'hashtag': row.hashtag, 'text': row.text,
                    'longitude': row.longitude, 'latitude':row.latitude, 'numberOfTweets': row.numberOfTweets, }
        eventRowsList.append(rowToAdd)

In [67]:
eventRowsList = []
count = 0
printingValue = int(len(dictionary) / 10)

pr('Starting to make event df with {} dataframes. (around 6 min)'.format(len(dictionary)))
for h, df in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df.apply(applyToMakeEventDf, axis=1)

pr('Making new dataframe.')
new_events = pd.DataFrame(eventRowsList)
new_events.set_index(['date'], inplace=True)
pr('Finished! Dataframe with {} rows'.format(len(new_events)))

19:10:07 Starting to make event df with 27040 dataframes. (around 6 min)
19:10:14 10%
19:10:19 20%
19:10:25 30%
19:10:31 40%
19:10:36 50%
19:10:42 60%
19:10:47 70%
19:10:52 80%
19:10:57 90%
19:11:03 100%
19:11:03 Making new dataframe.
19:11:03 Finished! Dataframe with 260 rows


In [68]:
print('Events dataframe:')
new_events.head(10)

Events dataframe:


,hashtag,latitude,longitude,numberOfTweets,text
date,,,,,
2012-05-10,physician,47.71160,9.458630,14,Arrow Electronics: Internal Product Special......
2011-06-08,ios5,47.48500,8.297210,9,@AndreBonhote einstellungen gemacht? #ios5 ht...
2011-10-02,fcz,47.38230,8.503910,13,"@DavidWiederkehr doch! Man kann, wenn man will..."
2011-09-11,osx,47.50000,8.750000,6,"Hm, whys #OSX not using the #VPN I've just now..."
2012-05-16,ilike,47.36870,7.989945,6,Freue mich jetzt schon auf die leckere Bratkar...
2012-05-07,switzerland,46.47450,6.909680,13,So good to be back #switzerland #lausanne @ S...
2012-05-09,switzerland,46.90690,8.045940,10,trip to geneva today :-) #switzerland_$$$_Luc...
2012-05-10,switzerland,47.04785,7.947230,10,OMG Lavaux! You're so effing beautiful!!! #Swi...
2012-05-17,switzerland,46.50615,6.630930,12,"#Switzerland: a new version of the good old ""s..."


In [69]:
print('Linked dataframe of all days:')
dictionary[new_events.iloc[0].hashtag].head(10)

Linked dataframe of all days:


,latitude,text,longitude,numberOfTweets,hashtag,event
2012-04-20,47.36730,#Physician #Job in #Zurich Rooms Systems Speci...,8.550000,1,physician,False
2012-04-22,47.71160,Arrow Electronics: Internal Product Special......,9.458630,1,physician,False
2012-04-25,47.36730,#Zurich #Physician #Job: Rooms Systems Special...,8.550000,1,physician,False
2012-05-04,47.53945,Arrow Electronics: Internal Product Special......,9.004315,2,physician,False
2012-05-09,47.36730,#Physician #Job alert: Rooms Systems Specialis...,8.550000,1,physician,False
2012-05-10,47.71160,Arrow Electronics: Internal Product Special......,9.458630,14,physician,True
2012-05-11,47.71160,Arrow Electronics: Internal Product Special......,9.458630,3,physician,False
2012-05-16,47.36730,Rooms Systems Specialist - Hyatt International...,8.550000,1,physician,False


## Exporting data

In [70]:
e_df = new_events.copy()
e_df['date'] = e_df.index
e_df.index = [i for i in range (len(e_df))]
e_df.head(1)

,hashtag,latitude,longitude,numberOfTweets,text,date
0,physician,47.7116,9.45863,14,Arrow Electronics: Internal Product Special......,2012-05-10


In [166]:
epoch_dt = datetime(1970, 1, 1)
def to_utc(date):
    d_dt = datetime.combine(date, datetime.min.time())
    return int((d_dt - epoch_dt).total_seconds()*1000)

In [ ]:
e_df['utc_date'] = e_df['date'].apply(lambda x: to_utc(x))

In [164]:
def convert_to_unix_time(record):
    datetime_index = pd.DatetimeIndex([datetime(record['year'], record['month'], 1)])
    unix_time_index = datetime_index.astype(np.int64) // 10**6
    return unix_time_index[0]

In [ ]:
e_df['unix_time'] = e_df.apply(convert_to_unix_time, axis=1)

In [165]:
e_df.head()

,hashtag,latitude,longitude,numberOfTweets,text,date,year,month,utc_date,unix_time
0,physician,47.7116,9.458630,14,Arrow Electronics: Internal Product Special......,2012-05-10,2012,5,1336608000000,1335830400000
1,ios5,47.4850,8.297210,9,@AndreBonhote einstellungen gemacht? #ios5 ht...,2011-06-08,2011,6,1307491200000,1306886400000
2,fcz,47.3823,8.503910,13,"@DavidWiederkehr doch! Man kann, wenn man will...",2011-10-02,2011,10,1317513600000,1317427200000
3,osx,47.5000,8.750000,6,"Hm, whys #OSX not using the #VPN I've just now...",2011-09-11,2011,9,1315699200000,1314835200000
4,ilike,47.3687,7.989945,6,Freue mich jetzt schon auf die leckere Bratkar...,2012-05-16,2012,5,1337126400000,1335830400000


In [167]:
e_gb_month = e_df.groupby(e_df.unix_time)

In [170]:
print('first {0} months '.format(len(e_gb_month.groups)))

first 18 months 


In [177]:
months = []
for month, df in e_gb_month:
    days = []
    for i in range (len(df)):
        ht = df.iloc[i]['hashtag']
        lat = df.iloc[i]['latitude']
        lon = df.iloc[i]['longitude']
        t_num = df.iloc[i]['numberOfTweets']
        tweets = df.iloc[i]['text'].split(delimiter)
        date = df.iloc[i]['utc_date']
        
        data_unit = { 'name': ht
                    , 'latitude' : lat
                    , 'longitude' : lon
                    , 'tweets' : tweets
                    , 'number_of_tweets' : str(t_num)
                    , 'date' : int(date)}
        days.append(data_unit)
    
    curr_month = {'date': int(month), 'data' : days}
    months.append(curr_month)

final_events = {'events' : months}

In [71]:
#e_gb = e_df.groupby(e_df.date)

In [74]:
'''events = []
pr('Making event list...')
for date, df in e_gb:
        units = []
        for i in range (len(df)):
            ht = df.iloc[i]['hashtag']
            lat = df.iloc[i]['latitude']
            lon = df.iloc[i]['longitude']
            t_num = df.iloc[i]['numberOfTweets']
            tweets = df.iloc[i]['text'].split(delimiter)
            
            data_unit = { 'name': ht
                         , 'latitude' : lat
                         , 'longitude' : lon
                         , 'tweets' : tweets
                         , 'number_of_tweets' : str(t_num) }
            units.append(data_unit)
        
        date_dict = {}
        #date_json = 'date: {0}'.format(date)
        #data_json = data_template.format(','.join(units))
        curr_event = {'date': date.isoformat(), 'data': units}
        events.append(curr_event)

final_events = {'events' : events}
pr('Finished.')'''

19:11:04 Making event list...
19:11:04 Finished.


In [178]:
pr('Exporting to json...')
with open('data/export.json', 'w') as f:
     json.dump(final_events, f)
pr('Export done.')

23:08:59 Exporting to json...
23:09:00 Export done.
